In [ ]:
#r "nuget: Newtonsoft.Json"
#r "nuget: Microsoft.Azure.Cosmos"
using Microsoft.Azure.Cosmos;
using System.Net.Http;

Azure Cosmos DB Emulator

GatewayModeMaxConnectionLimit

ConnectionMode

AllowBulkExecution

In [ ]:
var endpoint = "https://localhost:8081";
var key ="C2y6yDjf5/R+ob0N8A7Cgv30VRDJIWEHLM+4QDU5DE2nQ9nDuVTqobD4b8mGGyPMbIZnqyMsEcaGQy67XIw/Jw==";
var client = new CosmosClient(endpoint, key);

client.Display();

In [ ]:
CosmosClientOptions options = new CosmosClientOptions
{
    AllowBulkExecution = false,
    ApplicationName = "CosmosDBDemo",
    ApplicationRegion = "East US", 
    //threshold: how long before SDK begins hedging. thresholdStep: how much longer to wait before sending another request
    //hedging is a latency optimization strategy that sends requests to multiple regions and uses the first response
    AvailabilityStrategy = AvailabilityStrategy.CrossRegionHedgingStrategy(threshold: TimeSpan.FromMilliseconds(500), thresholdStep: TimeSpan.FromMilliseconds(100)),
    ConsistencyLevel = ConsistencyLevel.Session,
    EnableContentResponseOnWrite = false,    
    RequestTimeout = TimeSpan.FromSeconds(30),    
    ConnectionMode = ConnectionMode.Gateway,
    MaxRetryAttemptsOnRateLimitedRequests = 5,
    MaxRetryWaitTimeOnRateLimitedRequests = TimeSpan.FromSeconds(30)    
};

client = new CosmosClient(endpoint, key, options);
client.Display();

Meta data queries

In [ ]:
using (var iterator = client.GetDatabaseQueryIterator<DatabaseProperties>())
{
    while (iterator.HasMoreResults)
    {
        var response = await iterator.ReadNextAsync();
        foreach (var db in response)
        {
            Console.WriteLine($"Database Id: {db.Id}");
        }
    }
}

In [ ]:
var livedb = await client.CreateDatabaseIfNotExistsAsync("Live");
livedb.Display();

In [ ]:
var livedb = client.GetDatabase("Live");
var products = await livedb.CreateContainerIfNotExistsAsync("Products","/categoryid", 1000);
products.Display();

Rated
Auto Scale
Serverless

In [ ]:
var containerprops = new ContainerProperties
{
    Id = "Products2",
    PartitionKeyPath = "/categoryid"    
};

var throughputprops =  ThroughputProperties.CreateAutoscaleThroughput(4000);
var products = await livedb.CreateContainerIfNotExistsAsync(containerprops, throughputprops);
products.Display();


In [ ]:
var containertoupdate = livedb.GetContainer("Products2");
var autoscale = ThroughputProperties.CreateAutoscaleThroughput(8000);
var response = await containertoupdate.ReplaceThroughputAsync(autoscale);
response.Display();

#11:45